# Zero-Shot プロンプトを使用してテキスト生成用の Bedrock モデルを呼び出す

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 導入

このノートブックでは、サポートエンジニアから受けたカスタマーサービスの質について否定的なフィードバックを寄せた顧客に対して、LLM を使用して E メールによる返信を生成する方法を説明します。 

Beto3 APIを使用した Amazon Bedrock Titan Text Large モデルを使用します。 

この例で使用されているプロンプトは、期待する回答例を示していないため、Zero-Shot プロンプトと呼ばれています。

**Note:** *このノートブックは AWS 内外問わず実行することができます*

#### コンテキスト
Amazon Bedrock のテキスト生成機能を実証するために、Boto3 クライアントを使用して Amazon Bedrock API と通信する方法について見ていきます。さまざまな設定が可能であるとともに、単純な入力がいかに望ましい出力につながるかを説明します。

#### パターン
Amazon Bedrock API に、タスク、命令、入力を提供するだけで、追加の例は示さずに出力を生成できます。ここでの目的は、強力な LLM が目の前のタスクをいかに簡単に理解し、説得力のあるアウトプットを生成するかを示すことです。

![](./images/bedrock.jpg)

#### ユースケース
Amazon Bedrock のモデルの生成機能を実証するために、 E メール生成のユースケースを見てみましょう。

#### ペルソナ
あなたは AnyCompany のカスタマーサービスマネージャーである Bob ですが、一部の顧客はカスタマーサービスに満足しておらず、カスタマーサポートエンジニアが提供するサービスに否定的なフィードバックをしています。今度は、サービスの質が悪かったことを謙虚に謝罪する顧客に応え、信頼を取り戻したいと考えています。以前のメールでのやり取りから得た顧客の感情に合わせて、ヒューマンフレンドリーなメールを大量に生成するには、LLM の助けが必要です。

#### 実装
このユースケースを実現するために、このノートブックでは、お客様の以前のメールに基づいてサンキューノート付きのメールを生成する方法を示します。ここでは、Amazon Bedrock API と Boto3 クライアントを使用する Amazon Titan Text Large モデルを使用します。 

## セットアップ

このノートブックの以降を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrockに接続する必要があります。

セットアップの仕組みと ⚠️ **変更の要否** の詳細については [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ja.ipynb) を参照ください。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## テキスト生成

上記で説明したユースケースに続いて、Amazon Bedrock サービスが E メールを生成するための入力を準備しましょう。なお、このプロンプトは[Human:/Assistant: Formatting for Claude.](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting)で修正する必要があります。

In [ ]:
# プロンプトの作成
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


まず、Amazon Titan Text Large モデルを使用してみましょう。Amazon Titan Text Large は、最大 4 k トークンのコンテキストウィンドウをサポートし、以下のパラメータを受け入れます。
- `inputText`: LLM にインプットするプロンプト
- `textGenerationConfig`: モデルが出力を生成する際に考慮するパラメーター

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

Amazon Bedrock API には、以下を受け入れる API `invoke_model` が用意されています。
- `modelId`: Amazon Bedrock で入手可能なさまざまな基盤モデル ARN
- `accept`: インプットリクエストのタイプ
- `contentType`: アウトプットのコンテンツタイプ
- `body`: プロンプトと設定からなる JSON 文字列

Amazon Bedrock で利用可能なテキスト生成モデルには、次の ID があります。
- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v2`

#### Amazon Titan Text Large モデルの呼び出し

まず、以前に作成したプロンプトに基づいてモデルがどのように出力を生成するかを調べます。

##### 完全な出力生成

In [ ]:
modelId = 'amazon.titan-tg1-large' # 異なるモデルプロバイダーやバージョンを使用する場合は変更してください
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:
    
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# 応答の関連部分は最初の改行文字の後に始まります。
# 以下に、'\n' が最初に出現した時点から始まる応答を出力します

email = outputText[outputText.index('\n')+1:]
print_ww(email)


##### ストリーミング出力の生成
上記は、Amazon Titan Text Large モデルが、入力リクエストを理解し、さまざまなモダリティについて固有の理解に基づいて生成した E メールの例です。この API へのリクエストは同期的で、モデルによって出力全体が生成されるのを待ちます。

Bedrock は、モデルによって生成された出力をチャンクの形でストリーミングすることもサポートしています。以下は、ストリーミングオプションを使用してモデルを起動する例です。`invoke_model_with_response_stream` は `ResponseStream` を返し、そこから読み取ることができます。

In [ ]:
output = []
try:
    
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

上記は、モデルの出力をすばやく取得し、読みながらサービスに完成させるのに役立ちます。これは、モデルに生成するように要求するテキストが長くなるようなユースケースに役立ちます。生成されたすべてのチャンクを後で組み合わせて完全な出力を形成し、それをユースケースに使用することができます。

In [ ]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

## 結論
今回、Amazon Bedrock API の一般的な利用方法として、`boto3` SDK を試してみました。この API を使用して、顧客からの否定的なフィードバックに対する返信メールを生成するユースケースを見てきました。

### 発展要素
- Anthropic Claude や AI21 Labs の Jurassic モデルなど、Amazon Bedrock から入手できるさまざまなモデルを試すために、このノートブックを改造する
- プロンプトを特定のユースケースに変更し、さまざまなモデルの出力を評価する
- トークンの長さを複数変更して試すことで、サービスのレイテンシーと応答性を理解する
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を得る

## ありがとうございました